# MERFISH Data reorganization

In [1]:
import pandas as pd
import numpy as np
import os, sys, time
import re
import shutil
from importlib import reload
# this version doesn't have ChromAn compiled, so directly load from src:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares/ChromAn")
import src

In [2]:
from src.file_io.data_organization import search_fovs_in_folders

data_home =  r"/lab/weissman_imaging/puzheng"

In [9]:
data_folder = r"/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune"

fds, fovs = search_fovs_in_folders(data_folder)


- searching in folder: /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune
-- 9 folders, 255 fovs detected.


In [12]:
fds

['/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H1M2',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H3M4',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H5M6',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8',
 '/lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9']

In [10]:
date = os.path.basename(data_folder).split('-')[0]

sample = 'ms-Asw016'
library = 'MF1'

print(date, sample, library)

20230816 ms-Asw016 MF1


In [11]:
from src.file_io.data_organization import create_folder
# shutter files
ref_image_type = '748_637_545_477_405_s13' # ref 58 / 65
polyt_image_type = '748_637_545_477_s13'
image_type = '748_637_545_477_s13' # ref 36/39

# fiducial beads
#ref_channel = 488
#ref_frame = 13 # start from 0!

# target folder
merfish_data_folder = r'/lab/weissman_imaging/puzheng/MERFISH_data'

# target folder:
target_folder = os.path.join(merfish_data_folder, f'{date}-{sample}_{library}')
create_folder(target_folder)


Creating folder: /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1


In [19]:
#sel_fovs = []
folder_regexp = r'H([0-9]+)[MRCP]([0-9+])'

ref_round = 0
polyt_round = 8
#fov_ids = np.arange(2)
overwrite = True
remove_source = False

for _fov_id, _fov_name in enumerate(fovs):
    # check if this fov is selected:
    if 'sel_fovs' in locals() and _fov_id not in sel_fovs:
        continue # skip if not selected
    # prceed for the rest:
    for _fd in fds:
        # match round and hyb
        _match = re.search(folder_regexp, os.path.basename(_fd))
        if _match is None:
            continue
        _round, _region = _match.groups()
        # mutliple files for each image:
        _files = [os.path.join(_fd, _fl) for _fl in os.listdir(_fd) if _fov_name.split(os.extsep)[0] in _fl]
        # reference round specified
        if int(_round) == ref_round:
            _image_type = ref_image_type
        elif int(_round) == polyt_round:
            _image_type = polyt_image_type
        else:
            _image_type = image_type
    
        # target file
        _target_files = [os.path.join(target_folder, f"{_image_type}_{_fov_id}_{_round}{os.extsep}{_fl.split(os.extsep)[-1]}") 
                         for _fl in _files]
        for _fl, _tar_fl in zip(_files, _target_files):
            if remove_source:
                print(f"move {_fl} to {_tar_fl}")
                shutil.move(_fl, _tar_fl)
            else:
                if not os.path.isfile(_tar_fl) or overwrite:
                    print(f"copy {_fl} to {_tar_fl}")
                    shutil.copyfile(_fl, _tar_fl)
                else:
                    print(f"{_tar_fl} already exist, skip")

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_000.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_0_0.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_000.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_0_0.dax
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_0_0.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_0_0.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_000.power to /lab/w

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_0_8.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_0_8.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9/Conv_zscan_000.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_0_8.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_001.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_1_0.dax
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_001.inf to /lab/weissman_im

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_001.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_1_7.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_001.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_1_7.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_001.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_1_7.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_001.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_1_7.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9/Conv_zscan_001.dax to /lab/weissman_imagin

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7/Conv_zscan_002.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_2_6.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7/Conv_zscan_002.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_2_6.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7/Conv_zscan_002.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_2_6.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_002.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_2_7.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H7M8/Conv_zscan_002.dax to /lab/weissman_imagin

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H5M6/Conv_zscan_003.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_3_5.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H5M6/Conv_zscan_003.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_3_5.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H5M6/Conv_zscan_003.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_3_5.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7/Conv_zscan_003.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_3_6.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H6M7/Conv_zscan_003.dax to /lab/weissman_imagin

...

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_251.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_251_4.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_251.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_251_4.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_251.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_251_4.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_251.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_251_4.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_251.dax to /lab/weissma

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H3M4/Conv_zscan_252.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_252_3.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H3M4/Conv_zscan_252.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_252_3.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H3M4/Conv_zscan_252.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_252_3.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H3M4/Conv_zscan_252.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_252_3.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H4M5/Conv_zscan_252.dax to /lab/weissma

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3/Conv_zscan_253.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_253_2.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3/Conv_zscan_253.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_253_2.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3/Conv_zscan_253.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_253_2.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3/Conv_zscan_253.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_253_2.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H2M3/Conv_zscan_253.dax to /lab/weissma

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H0M1/Conv_zscan_254.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_405_s13_n1000_5Hz_254_0.xml
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H1M2/Conv_zscan_254.off to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_254_1.off
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H1M2/Conv_zscan_254.power to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_254_1.power
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H1M2/Conv_zscan_254.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_254_1.inf
copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H1M2/Conv_zscan_254.dax to /lab/wei

copy /lab/weissman_imaging/puzheng/msBrain/20230816-asw016_region_1_MF1msBrainImmune/H8M9/Conv_zscan_254.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20230816-ms-Asw016_MF1/748_637_545_477_s13_n1000_5Hz_254_8.xml


## Check file length

In [17]:
os.listdir(target_folder)

['748_637_545_477_405_s13_n1000_5Hz_24_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_70_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_174_0.xml',
 '748_637_545_477_s13_n1000_5Hz_32_8.dax',
 '748_637_545_477_s13_n1000_5Hz_213_8.off',
 '748_637_545_477_s13_n1000_5Hz_24_8.inf',
 '748_637_545_477_405_s13_n1000_5Hz_215_0.off',
 '748_637_545_477_405_s13_n1000_5Hz_45_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_219_0.off',
 '748_637_545_477_405_s13_n1000_5Hz_104_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_188_0.power',
 '748_637_545_477_405_s13_n1000_5Hz_199_0.inf',
 '748_637_545_477_s13_n1000_5Hz_61_8.power',
 '748_637_545_477_405_s13_n1000_5Hz_251_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_78_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_20_0.xml',
 '748_637_545_477_s13_n1000_5Hz_253_8.dax',
 '748_637_545_477_s13_n1000_5Hz_124_8.xml',
 '748_637_545_477_405_s13_n1000_5Hz_135_0.dax',
 '748_637_545_477_405_s13_n1000_5Hz_217_0.xml',
 '748_637_545_477_s13_n1000_5Hz_249_8.xml',
 '748_637_545_477_s1

In [20]:
# double check if filenumber equals to expected:
print(len(os.listdir(target_folder)), len(fds) * len(fovs) * 5)

11475 11475


# Generate data_organization

In [ ]:
## Automatic generate data_organization
from src.file_io.data_organization import Color_Usage

In [ ]:
def generate_dataOrganization():
    



In [ ]:
channelName	readoutName	imageType	imageRegExp	bitNumber	imagingRound	color	frame	zPos	fiducialImageType	fiducialRegExp	fiducialImagingRound	fiducialFrame	fiducialColor